In [1]:
!git clone https://github.com/vincentblot28/mapie_llm

Cloning into 'mapie_llm'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 22 (delta 3), reused 18 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 2.23 MiB | 3.86 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [2]:
import json
import sys
sys.path.append('/content/mapie_llm')

import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils.prompt_utils import TransformData


In [7]:
from huggingface_hub import notebook_login

notebook_login()

# Load data and transform it to X, y (X being a prompt)

In [8]:
data = json.load(open("/content/mapie_llm/cosmosqa_10k.json"))

In [9]:
data_transformer = TransformData(data)
X, y = data_transformer.transform_data()

# Load the model and its tokenizer

In [10]:
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
    model.eval()
    return tokenizer, model

# %%
tok, mo = load_model("mistralai/Mistral-7B-Instruct-v0.3")

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# Wrap the model

In [11]:
class LLMClassifier:


    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.options = ["Answer: A", "Answer: B", "Answer: C", "Answer: D"]
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.option_ids = [self.tokenizer.encode(opt)[-1] for opt in self.options]


    def predict_proba(self, prompts):
        if isinstance(prompts, str):
            prompts = [prompts]  # Convert single prompt to list
        pred_probas = []
        for prompt in prompts:
            prompt = [prompt]
            inputs = self._prepare_inputs(prompt)
            with torch.no_grad():
                outputs = self.model(**inputs)

            logits = outputs.logits[:, -1, :]  # Extract logits for the last token [batch_size, vocab_size]
            logits_options = logits[:, self.option_ids]  # Select logits corresponding to answer options

            probabilities = F.softmax(logits_options, dim=-1).cpu().numpy()
            pred_probas.append(probabilities)
        return probabilities

    def predict(self, prompts):
        probas = self.predict_proba(prompts)
        return np.argmax(probas, axis=-1)

    def _prepare_inputs(self, prompts):
        inputs = self.tokenizer(prompts, return_tensors="pt", truncation=True)
        for k in inputs:
            if torch.is_tensor(inputs[k]):
                inputs[k] = inputs[k].to("cuda")
        return inputs

In [12]:
llm = LLMClassifier(tok, mo)

In [13]:
llm.predict_proba(X[0]), llm.predict(X[0])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(array([[1.691e-03, 9.922e-01, 5.047e-03, 9.336e-04]], dtype=float16),
 array([1]))

In [14]:
y[0]

np.int64(1)